# Fair Classification on Adult Dataset

In [1]:
from pymoo.core.parameters import get_params

In [2]:
import copy
import pytest
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from fomo import FomoClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, average_precision_score
from pmlb import pmlb   
import fomo.metrics as metrics
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.algorithms.moo.nsga3 import NSGA3
from pymoo.util.ref_dirs import get_reference_directions
dataset = pmlb.fetch_data('adult')
# dataset = dataset.sample(n=2000)
X = dataset.drop('target',axis=1)
y = dataset['target']
Xtrain,Xtest, ytrain,ytest = train_test_split(X,y,
                                            stratify=y, 
                                            random_state=42,
                                            test_size=0.5
                                           )
ss = StandardScaler()
Xtrain = pd.DataFrame(ss.fit_transform(Xtrain), columns=Xtrain.columns, index=ytrain.index)
Xtest = pd.DataFrame(ss.transform(Xtest), columns=Xtest.columns, index=ytest.index)
groups = ['race','sex']

In [ ]:
from sklearn.metrics import make_scorer, accuracy_score
from fomo.problem import MLPProblem
est = FomoClassifier(
    estimator = LogisticRegression(),
    accuracy_metrics=[make_scorer(metrics.FPR)],
    fairness_metrics=[metrics.subgroup_FNR_scorer],
    verbose=True,
    problem_type=MLPProblem
)
est.fit(Xtrain,ytrain,protected_features=groups, termination=('n_gen',100))

running 8 processes
number of variables: 24421
number of objectives: 2


In [ ]:
est.plot().show()

# visualize model set

`est.plot()` will return the Pareto front found during the run, with a red dot indicating the final chosen model. 
By default, Fomo uses the PseudoWeights method from pymoo to choose the final model, which produces a model near the centroid of the front. 

In [ ]:
est.plot().show()

## picking with different strategies

We can also pick with other multi-criteria decision making strategies (MDCMs). 
Fomo supports the PseudoWeights, Compromise, and HighTradeoffPoints strategies from [pymoo](https://pymoo.org/mcdm). 

Here's an example of picking with the Compromise strategy.

In [ ]:
est.best_estimator_ = est.pick_best('PseudoWeights')
plot = est.plot()
plot.show()
plt.title('Picking with PseudoWeights')


In [ ]:
est.best_estimator_ = est.pick_best('Compromise')
plot2 = est.plot()
plot2.show()
plt.title('Picking with Compromise')


Here's an example of picking with the HighTradeoffPoints strategy.

In [ ]:
est.best_estimator_ = est.pick_best('HighTradeoffPoints')
plot3 = est.plot()
plot3.show()
plt.title('Picking with HighTradeoffPoints')